In [1]:
import tensorflow_datasets as tfds
imdb,info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDSDRI0/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDSDRI0/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [2]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [3]:
print(imdb)

{'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [4]:
for i in imdb['train'].take(9):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [5]:
import numpy as np

r,e = imdb['train'],imdb['test']

rs=[]
rl=[]
es=[]
el=[]

for s,l in r:
  rs.append(s.numpy().decode('utf8'))
  rl.append(l.numpy())

for s,l in e:
  es.append(s.numpy().decode('utf8'))
  el.append(l.numpy())

rlf = np.array(rl)
elf = np.array(el)

In [6]:
vs = 10000
ml = 120
ed = 16
tt = 'post'
ot = '<x>'

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

t = Tokenizer(num_words=vs,oov_token=ot)
t.fit_on_texts(rs)
w = t.word_index

q = t.texts_to_sequences(rs)
p = pad_sequences(q,maxlen=ml,truncating=tt)

eq = t.texts_to_sequences(es)
ep = pad_sequences(eq,maxlen=ml,truncating=tt)


In [9]:
import tensorflow as tf

m = tf.keras.Sequential([
    tf.keras.layers.Embedding(vs,ed,input_length=ml),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

m.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [10]:
ne = 10
m.fit(p,rlf,epochs=ne,validation_data=(ep,elf))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.4880 - accuracy: 0.7486 - val_loss: 0.3734 - val_accuracy: 0.8327
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2368 - accuracy: 0.9079 - val_loss: 0.4107 - val_accuracy: 0.8203
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0891 - accuracy: 0.9769 - val_loss: 0.5082 - val_accuracy: 0.8125
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0207 - accuracy: 0.9973 - val_loss: 0.6108 - val_accuracy: 0.8084
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0046 - accuracy: 0.9998 - val_loss: 0.6703 - val_accuracy: 0.8117
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.7380 - val_accuracy: 0.8120
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 8.3658e-04 - accuracy: 1.0000 - val_loss: 0.7773 - val_accuracy: 0.8134
Ep

In [11]:
el = m.layers[0]
ew = el.get_weights()[0]

rwi = t.index_word
ew.shape

(10000, 16)

In [12]:
import io
ov = io.open('vecs.tsv','w',encoding='utf-8')
om = io.open('meta.tsv','w',encoding='utf-8')

for w in range(1,vs):
  wn = rwi[w]
  we = ew[w]

  om.write(wn+'\n')
  ov.write('\t'.join([str(x) for x in we])+'\n')
  
om.close()
ov.close()

In [13]:
from google.colab import files
files.download('vecs.tsv')
files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>